# Inner join panos vs nott reference

In [1]:
library("tidyverse")

── Attaching core tidyverse packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


## Load data

In [2]:
panos = read_delim("./panos_annotation/ABC_results_ASSOCIATIONS_ALL_CPM1_RK_12_2_20.csv.gz")

Rows: 7412021 Columns: 24
── Column specification ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr  (6): chr, TargetGene, TargetGene_name, PeakID, PeakID_seqnames, combo
dbl (18): start, end, activity_base, TargetGeneTSS, TargetGenePromoterActivi...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
mic_enh = read.delim("microglia_hg38.Enh.Nott.tsv")

In [4]:
mic_prom = read.delim("microglia_hg38.Prom.Nott.tsv")

## Intersection region

In [5]:
panos_region = panos%>%select(chr,start,end)%>%unique

In [6]:
panos_enh = inner_join(panos_region,mic_enh, by = "chr")

Warning message in inner_join(panos_region, mic_enh, by = "chr"):
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 1 of `x` matches multiple rows in `y`.
ℹ Row 1 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship = "many-to-many"` to silence this warning.”


In [11]:
mic_enh%>%filter(start == 0 | end == 0)

X,chr,start,end
<int>,<chr>,<int>,<int>
2284,chr1,0,144925589
2300,chr1,149103586,0
2358,chr1,0,148319031
17793,chr17,0,0
17794,chr17,0,0
17795,chr17,0,0
17796,chr17,0,0
17797,chr17,0,0
21226,chr19,0,39898671


In [13]:
panos_enh = panos_enh%>%filter((end.x > start.y & end.x < end.y) | # ABC right overlap  with Annot left
                   (end.y > start.x & end.y < end.x  ) | # ABC left overlap  with Annot right
                   (start.y > start.x & end.y < end.x  ) | # Annot contained by  ABC 
                    (start.x > start.y & end.x < end.y  )   # ABC contained by  Annot 
                   )%>%filter(start.y != 0 , end.y != 0 ) # Rid of annot failure

In [16]:
panos_enh%>%write_delim("ABC_overlap_Annot_Enh.tsv","\t")

In [17]:
panos_prom  = inner_join(panos_region,mic_prom, by = "chr")%>%filter((end.x > start.y & end.x < end.y) | # ABC right overlap  with Annot left
                   (end.y > start.x & end.y < end.x  ) | # ABC left overlap  with Annot right
                   (start.y > start.x & end.y < end.x  ) | # Annot contained by  ABC 
                    (start.x > start.y & end.x < end.y  )   # ABC contained by  Annot 
                   )%>%filter(start.y != 0 , end.y != 0 ) # Rid of annot failure

Warning message in inner_join(panos_region, mic_prom, by = "chr"):
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 1 of `x` matches multiple rows in `y`.
ℹ Row 1 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship = "many-to-many"` to silence this warning.”


In [18]:
panos_prom%>%write_delim("ABC_overlap_Annot_prom.tsv","\t")

In [21]:
panos_region_overlapped = rbind(panos_enh%>%mutate(type = "Enh"), panos_prom%>%mutate(type = "Prom"))

In [22]:
panos_region_overlapped%>%write_delim("ABC_overlap_Annot_overlap.tsv","\t")

In [36]:
panos_region_overlapped%>%filter(chr =="chr15", start.x > 33000000, end.x < 34000000 )

chr,start.x,end.x,X,start.y,end.y,type
<chr>,<dbl>,<dbl>,<int>,<int>,<int>,<chr>
chr15,33201739,33202238,13884,33201604,33202212,Enh
chr15,33150992,33151491,13880,33150482,33156242,Enh
chr15,33152434,33152933,13880,33150482,33156242,Enh
chr15,33022503,33023002,13877,33021175,33023225,Enh
chr15,33005502,33006001,13876,33004966,33009295,Enh
chr15,33007272,33007771,13876,33004966,33009295,Enh
chr15,33158315,33158814,13882,33158641,33158951,Enh
chr15,33029429,33029928,13878,33029188,33029964,Enh
chr15,33066606,33067105,3811,33065785,33068812,Prom


In [34]:
panos %>%filter( start %in% (panos_region_overlapped%>%filter(chr =="chr15", start.x > 33000000, end.x < 34000000 ))$start.x, TargetGene_name == "FMN1" )

chr,start,end,activity_base,TargetGene,TargetGene_name,TargetGeneTSS,TargetGenePromoterActivityQuantile,distance,PeakID,⋯,ABC.Score,powerlaw_contact,powerlaw_contact_reference,hic_contact,hic_contact_pl_scaled,hic_pseudocount,hic_contact_pl_scaled_adj,ABC.Score.Numerator,powerlaw.Score.Numerator,powerlaw.Score
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr15,33201739,33202238,1.411447,ENSG00000248905,FMN1,33194733,0.899655,7255.5,Peak_61095,⋯,0.054683,0.089287,0.089287,0.067544,0.067544,0.001229,0.068773,0.097069,0.126024,0.080638
chr15,33150992,33151491,2.223208,ENSG00000248905,FMN1,33194733,0.899655,43491.5,Peak_61088,⋯,0.026866,0.018802,0.018802,0.020222,0.020222,0.001229,0.021451,0.047690,0.041800,0.026746
chr15,33152434,33152933,1.969647,ENSG00000248905,FMN1,33194733,0.899655,42049.5,Peak_61089,⋯,0.023802,0.019362,0.019362,0.020222,0.020222,0.001229,0.021451,0.042251,0.038135,0.024401
chr15,33158315,33158814,0.721119,ENSG00000248905,FMN1,33194733,0.899655,36168.5,Peak_61091,⋯,0.009726,0.022073,0.022073,0.022713,0.022713,0.001229,0.023942,0.017265,0.015917,0.010185
chr15,33005502,33006001,2.377608,ENSG00000248905,FMN1,33194733,0.899655,188981.5,Peak_61073,⋯,0.008793,0.005238,0.005238,0.005336,0.005336,0.001229,0.006565,0.015608,0.012453,0.007968
chr15,33022503,33023002,2.183530,ENSG00000248905,FMN1,33194733,0.899655,171980.5,Peak_61075,⋯,0.008576,0.005685,0.005685,0.005743,0.005743,0.001229,0.006972,0.015224,0.012414,0.007943
chr15,33007272,33007771,1.644320,ENSG00000248905,FMN1,33194733,0.899655,187211.5,Peak_61074,⋯,0.006081,0.005281,0.005281,0.005336,0.005336,0.001229,0.006565,0.010795,0.008683,0.005556
chr15,33066606,33067105,0.954817,ENSG00000248905,FMN1,33194733,0.899655,127877.5,Peak_61080,⋯,0.004698,0.007357,0.007357,0.007504,0.007504,0.001229,0.008733,0.008339,0.007025,0.004495
chr15,33029429,33029928,0.715952,ENSG00000248905,FMN1,33194733,0.899655,165054.5,Peak_61076,⋯,0.002873,0.005892,0.005892,0.005894,0.005894,0.001229,0.007123,0.005100,0.004219,0.002699


In [65]:
panos %>%colnames

[1] "chr"                                "start"                             
 [3] "end"                                "activity_base"                     
 [5] "TargetGene"                         "TargetGene_name"                   
 [7] "TargetGeneTSS"                      "TargetGenePromoterActivityQuantile"
 [9] "distance"                           "PeakID"                            
[11] "PeakID_seqnames"                    "PeakID_start"                      
[13] "PeakID_end"                         "combo"                             
[15] "ABC.Score"                          "powerlaw_contact"                  
[17] "powerlaw_contact_reference"         "hic_contact"                       
[19] "hic_contact_pl_scaled"              "hic_pseudocount"                   
[21] "hic_contact_pl_scaled_adj"          "ABC.Score.Numerator"               
[23] "powerlaw.Score.Numerator"           "powerlaw.Score"

### Write Panos ATAT-Seq X Nott as bed.

In [63]:
panos_region_overlapped%>%unique%>%
select(chr,start = start.x, end = end.x, type)%>%unique%>%
mutate(chr = str_remove(chr, "chr")%>%
       factor(levels = c(1:22,"X","Y")))%>%
arrange(chr,start,end)%>%
mutate(type = paste0(type,":",as.character(chr),":",start,":",end ))%>%
write_delim("./panos_annotation/Panos_overlaped_Nott2019_LHX2_H3K27ac_TableS5_prom_enh_hg38.bed","\t",col_names = FALSE)

### Write Panos ATAT-Seq as bed.

In [70]:
panos_bed = panos%>%select(chr,start,end,PeakID,activity_base)%>%unique

In [74]:
panos_bed%>%unique%>%
mutate(chr = str_remove(chr, "chr")%>%
       factor(levels = c(1:22,"X","Y")))%>%
arrange(chr,start,end)%>%
mutate(type = paste0(PeakID,":",activity_base,":",as.character(chr),":",start,":",end ))%>%
select(-PeakID,-activity_base)%>%
write_delim("./panos_annotation/Panos_2023_ABC_region.bed","\t",col_names = FALSE)

### Write Microglia Nott as bed.

In [73]:
rbind(mic_enh%>%mutate(type = "Enh"),mic_prom%>%mutate(type = "Prom"))%>%
    mutate(chr = str_remove(chr, "chr")%>%
       factor(levels = c(1:22,"X","Y")))%>%
arrange(chr,start,end)%>%
mutate(type = paste0(type,":",as.character(chr),":",start,":",end ))%>%
select(-X)%>%
write_delim("./panos_annotation/Nott2019_Mic_H3K27ac_TableS5_prom_enh_hg38.bed","\t",col_names = FALSE)


In [ ]:
panos_region_overlapped%>%unique%>%
select(chr,start = start.x, end = end.x, type)%>%unique%>%
mutate(chr = str_remove(chr, "chr")%>%
       factor(levels = c(1:22,"X","Y")))%>%
arrange(chr,start,end)%>%
mutate(type = paste0(type,":",as.character(chr),":",start,":",end ))%>%
write_delim("./panos_annotation/Panos_overlaped_Nott2019_LHX2_H3K27ac_TableS5_prom_enh_hg38.bed","\t",col_names = FALSE)

In [47]:
bed = read_delim("/sc/arion/projects/load/users/patelt16/projects/regulatory_gwas/20230429/wgs.reg.vcf_to_gds/input/Nott2019_LHX2_H3K27ac_TableS5_prom_enh_hg38.bed")

Rows: 64176 Columns: 4
── Column specification ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): 1, ENH:1:817069:817440
dbl (2): 817069, 817440

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [55]:
bed

1,817069,817440,ENH:1:817069:817440
<chr>,<dbl>,<dbl>,<chr>
1,818117,818345,ENH:1:818117:818345
1,818682,818799,ENH:1:818682:818799
1,904119,905001,ENH:1:904119:905001
1,910178,910472,ENH:1:910178:910472
1,939920,940068,ENH:1:939920:940068
1,976023,976683,ENH:1:976023:976683
1,976912,977020,ENH:1:976912:977020
1,997336,997471,ENH:1:997336:997471
1,998014,998404,ENH:1:998014:998404
